<a href="https://colab.research.google.com/github/anne-urai/ddm_mediation/blob/main/fit_simdata_HDDMnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, set Runtime -> Runtime type -> GPU

# https://hddm.readthedocs.io/en/latest/lan_tutorial.html
!pip install scikit-learn
!pip install cython
!pip install pymc

!pip install -U --no-deps git+https://github.com/hddm-devs/hddm
!pip install -U --no-deps git+https://github.com/hddm-devs/kabuki

  Cloning https://github.com/hddm-devs/hddm to /tmp/pip-req-build-ldd2rg8g
  Running command git clone -q https://github.com/hddm-devs/hddm /tmp/pip-req-build-ldd2rg8g
  Running command git submodule update --init --recursive -q
  Cloning https://github.com/hddm-devs/kabuki to /tmp/pip-req-build-kwnnt955
  Running command git clone -q https://github.com/hddm-devs/kabuki /tmp/pip-req-build-kwnnt955


In [ ]:
# MODULE IMPORTS ----
# warning settings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data management
import pandas as pd
import numpy as np
import pickle

# Plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Stats functionality
from statsmodels.distributions.empirical_distribution import ECDF

# to download data
from google.colab import files
import pprint

# HDDM
import hddm

SyntaxError: ignored

In [ ]:
##  ============================================================= ##
## generate some artificial choice data with mediation variable
##  ============================================================= ##

# first, previous choices ('X')
n_trials = 500
n_subj = 20
df = pd.DataFrame(np.random.choice([-1,1], (n_trials*n_subj, 2)), 
                  columns=list('XS')) 

# then a normally distributed random variable, that depends on X
df['M'] = np.random.normal(size=n_trials*n_subj) + df['X']

# make sure we have subj_idx and trial_idx
subj_idx = []
for s in np.arange(n_subj) + 1:
  for n in np.arange(n_trials):
    subj_idx.append(s)

trial_idx = []
for s in np.arange(n_subj) + 1:
  for n in np.arange(n_trials):
    trial_idx.append(n)

df['subj_idx'] = subj_idx
df['trial_idx'] = trial_idx

df = df[['subj_idx', 'trial_idx', 'S', 'X', 'M']]
df.sample(n=20)


In [ ]:
# see the separation
g = sns.FacetGrid(data=df, col='subj_idx', col_wrap=5, hue='X')
g.map(sns.distplot, 'M')

In [ ]:
##  ============================================================= ##
# plug into HDDMnn to simulate choices and RTs
##  ============================================================= ##

# help(hddm.simulators.hddm_dataset_generators.simulator_h_c)
# see https://hddm.readthedocs.io/en/latest/lan_tutorial.html#section-5-regressors 

model = 'ddm' # start simple
data, full_parameter_dict = hddm.simulators.hddm_dataset_generators.simulator_h_c(data = df, 
                                                                                  model = model,
                                                                                  p_outlier = 0.00,
                                                                                  conditions = None, #{'X': [-1, 1], 'S':[-1,1]}, 
                                                                                  depends_on = None, #{'v': ['X','S']},
                                                                                  regression_models = ['v ~ 1 + S + X'], 
                                                                                  #regression_covariates = ['S', 'X'],
                                                                                  regression_covariates = {'S': {'type': 'categorical', 'range': (-1, 1)},
                                                                                                           'X': {'type': 'categorical', 'range': (-1, 1)}},
                                                                                  group_only = None,
                                                                                  group_only_regressors = False,
                                                                                  fixed_at_default = None)


In [ ]:
# help(hddm.simulators.hddm_dataset_generators.simulator_h_c)
data
hddm.simulators.model_config[model]["params"]


In [ ]:
range(1)

In [ ]:
# param_dict = {k: [x for x in v if 'reg' not in x] for k, v in full_parameter_dict.items()}
param_dict = {}
for key, value in full_parameter_dict.items():
    if 'reg' not in key:
        param_dict[key] = value
param_dict#
param_df = pd.DataFrame.from_dict(param_dict, orient='index').reset_index()
param_df


In [ ]:
key

In [ ]:
# see https://colab.research.google.com/notebooks/io.ipynb#scrollTo=p2E4EKhCWEC5
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

In [ ]:
# which HDDMnn models exist?
import pprint
mm = hddm.model_config.model_config.keys()
pprint.pprint(list(mm))
# for m in mm:
#   print('\n\n' + m)
#   pprint.pprint(hddm.model_config.model_config[m])

In [ ]:
# Make HDDM Model to FIT

# Set up the regressor a regressor:
reg_model_v = {'model': 'v ~ 1 + stimulus + prevresp', 'link_func': lambda x: x}
reg_model_z = {'model': 'z ~ 1 + prevresp', 'link_func': lambda x: x}
reg_descr = [reg_model_z, reg_model_v]

hddmnn_model = hddm.HDDMnn(data,
                           model = model,
                           informative = False,
                           include = hddm.simulators.model_config[model]['hddm_include'],
                           p_outlier = 0.05,
                           is_group_model = True)
hddmnn_model.sample(nmcmc, burn = 100)

In [ ]:
# Caterpillar Plot: (Parameters recovered ok?)
hddm.plotting.plot_caterpillar(hddm_model = hddmnn_model,
                               ground_truth_parameter_dict = full_parameter_dict,
                               figsize = (8, 8),
                               columns = 3)
plt.show()